In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import robot as Robot
import  time

robot_Instance= Robot.Robot_Class(6000,1000,90)

In [4]:
Start=time.time()
while((robot_Instance.Wait_For_A_Message(0.2))!=None):
    pass
robot_Instance.Set_Robot_Position(0,0,0,True)
robot_Instance.Clear_Read_Buffer()
robot_Instance.Go_To(1000,0,0)
print(time.time()-Start)

Going to 3000   0 with R 0 level 0
('G0 X3000 Y0 T4800 R0\r\n', 'M201 H0 S750\r\n', 'M3 H3\r\n')
{'h': 0, 'd': 0, 'a': 0, 'b': 0, 'l': 1, 'k': 0, 'j': 0, 'f': 0, 'c': 0, 'g': 0, 'm': 0, 'e': 0, 'i': 0}
 M0 X427.25 Y8.29 A1.17 T2 S2048

Blocked
Front go to left 421.12432007155985 308.22745355559437
Going to 421.12432007155985   308.22745355559437 with R 0 level 1
('G0 X421.0 Y308.0 T1000 R0\r\n', 'M201 H0 S750\r\n', 'M3 H3\r\n')
{'h': 0, 'd': 0, 'a': 0, 'b': 0, 'l': 1, 'k': 0, 'j': 0, 'f': 0, 'c': 0, 'g': 0, 'm': 0, 'e': 0, 'i': 0}
 M0 X431.64 Y6.61 A11.95 T1 S2048

Timeout
End of 1
('G0 X3000 Y0 T4800 R0\r\n', 'M201 H0 S750\r\n', 'M3 H3\r\n')
{'h': 0, 'd': 0, 'a': 0, 'b': 0, 'l': 1, 'k': 0, 'j': 0, 'f': 0, 'c': 0, 'g': 0, 'm': 0, 'e': 0, 'i': 0}
 M0 X426.42 Y5.73 A4.86 T2 S2048

Blocked
Front go to left 401.003603357378 304.65140569337774
Going to 401.003603357378   304.65140569337774 with R 0 level 1
('G0 X401.0 Y305.0 T1000 R0\r\n', 'M201 H0 S750\r\n', 'M3 H3\r\n')
{'h': 0, 'd': 0, '

In [ ]:
robot_Instance.Wait_For_A_Message(0.2)

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/pi/.local/lib/python3.5/site-packages/serial/serialposix.py", line 501, in read
    'device reports readiness to read but returned no data '
serial.serialutil.SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pi/STI_Robotic_Competition_Software/Architecture/Serial_.py", line 32, in read_from_port
    reading = self.ser.readline().decode('utf-8')
  File "/home/pi/.local/lib/python3.5/site-packages/serial/serialposix.py", line 509, in read
    raise SerialException('read failed: {}'.format(e))
serial.serialutil.SerialException: read fa